<a href="https://colab.research.google.com/github/chaemini-nimini/first-project-beethoven/blob/main/%EB%B2%A0%ED%86%A0%EB%B2%A4_%EB%9E%9C%EB%8D%A4_%EC%9E%AC%EC%83%9D%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pyngrok이 설치되어 있지 않은 경우를 대비하여 설치 명령어를 추가합니다.
# 런타임 재시작 시 모듈이 사라질 수 있으므로, 매번 실행되도록 합니다.
!pip install pyngrok

import os
from flask import Flask, send_from_directory, jsonify
import urllib.parse
import random
import subprocess
import time
import json
from pyngrok import ngrok, conf

# --- Google Drive 마운트 ---
try:
    from google.colab import drive
    print("Google Drive 마운트 중...")
    drive.mount('/content/drive')
    print("Google Drive 마운트 완료.")
except Exception as e:
    print(f"Google Drive 마운트 중 오류 발생: {e}")
    print("Google Colab 환경이 아니거나, 이미 마운트되어 있을 수 있습니다.")
    print("만약 Drive가 필요한 환경이라면, 수동으로 'from google.colab import drive; drive.mount('/content/drive')'를 실행해주세요.")

app = Flask(__name__)

# --- ngrok 실행 파일 다운로드 및 설정 ---
ngrok_path = "/usr/local/bin/ngrok"
if not os.path.exists(ngrok_path):
    print("ngrok 실행 파일 다운로드 중...")
    try:
        subprocess.run(["wget", "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip"], check=True, capture_output=True)
        subprocess.run(["unzip", "-o", "ngrok-stable-linux-amd64.zip"], check=True, capture_output=True)
        subprocess.run(["mv", "ngrok", ngrok_path], check=True, capture_output=True)
        subprocess.run(["chmod", "+x", ngrok_path], check=True, capture_output=True)
        print("ngrok 다운로드 및 설정 완료.")
    except Exception as e:
        print(f"ngrok 설정 중 오류 발생: {e}. ngrok 수동 설치가 필요할 수 있습니다.")
        ngrok_path = None
else:
    print("ngrok 실행 파일이 이미 존재합니다.")

# --- ngrok 인증 토큰 설정 (필수) ---
NGROK_AUTH_TOKEN = "2zH1ogBnCwMM25wNykF3dGIHZbv_AN1TLC5aiBfduuyLLi6w"

if NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "YOUR_NGROK_AUTH_TOKEN_HERE":
    try:
        print("ngrok 인증 토큰 설정 중...")
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        print("ngrok 인증 토큰 설정 완료.")
    except Exception as e:
        print(f"ngrok 인증 토큰 설정 중 오류 발생: {e}. 토큰을 다시 확인해주세요.")
else:
    print("경고: ngrok 인증 토큰이 설정되지 않았습니다. ngrok 터널이 작동하지 않을 수 있습니다.")
    print("https://dashboard.ngrok.com/get-started/your-authtoken 에서 토큰을 얻어 'NGROK_AUTH_TOKEN' 변수에 입력해주세요.")

# --- ngrok 터널 시작 및 URL 획득 (pyngrok 사용) ---
public_url = None
if ngrok_path and NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "YOUR_NGROK_AUTH_TOKEN_HERE":
    try:
        print("ngrok 터널 시작 중 (pyngrok 사용)...")
        conf.get_default().ngrok_path = ngrok_path
        http_tunnel = ngrok.connect(5000, "http")
        public_url = http_tunnel.public_url
        print(f"ngrok 터널 URL: {public_url}")
    except Exception as e:
        print(f"ngrok 터널 시작 중 오류 발생: {e}. ngrok 서비스에 문제가 있을 수 있습니다.")
        public_url = None
else:
    print("ngrok 실행 파일 또는 인증 토큰 문제로 터널을 시작할 수 없습니다.")

# --- 사용자 설정: Google Drive에 마운트된 폴더 경로를 여기에 입력하세요 ---
folder_path = '/content/drive/MyDrive/새 폴더' # 음악 파일 경로
# -------------------------------------------------------------------------

# --- 사용자 설정: 배경 이미지 파일명 (Google Drive 폴더 내) ---
background_image_filename = '160504-salvador-dali-01.webp' # 배경 이미지 파일
# -----------------------------------------------------------------

music_files = []
all_files_in_folder = []

# Flask 앱 시작 전 파일 존재 여부 및 목록 로드
try:
    if not os.path.exists(folder_path):
        print(f"오류: 지정된 폴더 경로 '{folder_path}'가 존재하지 않습니다.")
        print("Google Drive 마운트 상태와 폴더 경로를 다시 확인해주세요.")
    else:
        all_files_in_folder = os.listdir(folder_path)
        print(f"\n--- '{folder_path}' 폴더 내 모든 파일: {all_files_in_folder} ---")

        for filename in all_files_in_folder:
            # MP3 파일만 추가하도록 확장자 확인
            if filename.lower().endswith(".mp3"): # 대소문자 구분 없이 .mp3 확인
                music_files.append(filename)
        print(f"--- {len(music_files)}개의 MP3 음악 파일이 로드되었습니다. 로드된 파일: {music_files} ---")

        if not music_files:
            print("경고: 지정된 폴더에 MP3 파일이 없습니다. 음악이 재생되지 않을 수 있습니다.")
            print("음악 파일이 실제로 이 경로에 있는지, 확장자가 .mp3인지 확인해주세요.")

        if background_image_filename not in all_files_in_folder:
            print(f"경고: 배경 이미지 파일 '{background_image_filename}'이(가) '{folder_path}'에 없습니다.")
            print("파일명을 정확히 확인하고 (대소문자 구분), 해당 폴더에 파일을 업로드해주세요.")
except Exception as e:
    print(f"음악 파일 로드 중 치명적인 오류 발생: {e}")
    music_files = [] # 오류 발생 시 목록 초기화

@app.route('/')
def home():
    """
    이 경로는 웹 애플리케이션의 메인 HTML 페이지를 제공합니다.
    사용자 인터페이스 (버튼, 음악 제목 표시 등) 및 JavaScript 로직이 포함됩니다.
    """
    # 배경 이미지 파일명을 URL 인코딩합니다.
    encoded_bg_filename = urllib.parse.quote(background_image_filename)
    # 배경 이미지의 URL 경로를 생성합니다.
    bg_image_url = f"/music/{encoded_bg_filename}"

    # Google Drive에 저장한 재생 버튼 이미지 경로를 '새 폴더' 내부로 지정합니다.
    # 이 경로가 정확한지 다시 한번 확인해주세요.
    play_button_image_path = '/content/drive/MyDrive/새 폴더/Gemini_Generated_Image_3rnjgm3rnjgm3rnj.png'

    # Flask에서 정적 파일로 제공하기 위한 URL 경로를 생성합니다.
    encoded_play_button_filename = urllib.parse.quote(os.path.basename(play_button_image_path))
    play_button_image_url = f"/static_image/{encoded_play_button_filename}"


    # HTML/CSS/JavaScript 내의 모든 리터럴 {}는 {{}}로 이스케이프합니다.
    return '''
    <!DOCTYPE html>
    <html lang="ko">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Beethoven 음악 재생기</title>
        <script src="https://cdn.tailwindcss.com"></script>
        <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap" rel="stylesheet">
        <style>
            body {{
                font-family: 'Inter', sans-serif;
                background-image: url('{bg_image_url}');
                background-size: cover;
                background-position: center;
                background-attachment: fixed;
                background-color: #1a202c;
                position: relative;
                overflow: hidden; /* 스크롤바 방지 */
            }}
            /* 메인 컨테이너 (이전 'Beethoven' 박스) - 완전히 숨김 */
            #app {{
                display: none; /* 이 요소를 완전히 숨깁니다 */
            }}

            /* 재생 버튼 스타일 (이미지 사용, 오른쪽 적당한 곳으로 조정) */
            #playButton {{
                position: absolute;
                bottom: 50px; /* 아래에서 50px 위로 */
                right: 50px; /* 오른쪽에서 50px 왼쪽으로 */
                width: 150px; /* 이미지 크기 조절 */
                height: 150px; /* 이미지 크기 조절 */
                background: none; /* 배경색 제거 */
                border: none;
                cursor: pointer;
                padding: 0;
                border-radius: 20%; /* 사각형 이미지일 경우 약간 둥글게 */
                overflow: hidden; /* 이미지가 버튼 영역을 넘치지 않도록 */
                box-shadow: 0 8px 25px rgba(0, 0, 0, 0.4); /* 그림자 강화 */
                transition: transform 0.2s ease-in-out, box-shadow 0.2s ease-in-out;
                z-index: 100; /* 다른 요소 위에 표시 */
            }}
            #playButton:hover {{
                transform: scale(1.08); /* 호버 시 약간 확대 */
                box-shadow: 0 12px 30px rgba(0, 0, 0, 0.5); /* 호버 시 그림자 더 강화 */
            }}
            #playButton:active {{
                transform: scale(0.95);
            }}
            #playButton img {{
                display: block;
                width: 100%;
                height: 100%;
                object-fit: cover; /* 이미지가 버튼 영역을 꽉 채우도록 */
                border-radius: inherit; /* 부모의 border-radius 상속 */
            }}
             /* 음악 제목 표시를 위한 새로운 요소 (버튼 위에 띄울 수 있음) */
            #songTitleOverlay {{
                position: absolute;
                top: 20px; /* 상단에서 20px 아래로 */
                left: 50%;
                transform: translateX(-50%);
                color: white;
                font-size: 1.5rem;
                font-weight: bold;
                text-shadow: 2px 2px 5px rgba(0,0,0,0.7);
                z-index: 10;
            }}
            #loadingIndicatorOverlay, #errorMessageOverlay {{
                position: absolute;
                top: 80px; /* 제목 아래에 표시 */
                left: 50%;
                transform: translateX(-50%);
                z-index: 10;
                color: white; /* 로딩/에러 메시지 색상 */
                background-color: rgba(0,0,0,0.5);
                padding: 5px 10px;
                border-radius: 5px;
            }}
        </style>
    </head>
    <body class="min-h-screen flex items-center justify-center p-4">
        <div id="app">
            <h1 class="text-4xl md:text-5xl font-extrabold mb-6 drop-shadow-lg">Beethoven</h1>
            <p id="currentSong" class="text-lg md:text-xl font-medium mb-8 opacity-90 min-h-[2.5rem] flex items-center justify-center">
                재생할 음악을 선택해주세요.
            </p>
            <audio id="audioPlayer" class="hidden"></audio>
            <div id="loadingIndicator" class="mt-4 text-sm opacity-70 hidden">
                <svg class="animate-spin h-5 w-5 mr-3 inline-block text-white" viewBox="0 0 24 24">
                    <circle class="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" stroke-width="4"></circle>
                    <path class="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path>
                </svg>
                음악 로딩 중...
            </div>
            <div id="errorMessage" class="mt-4 text-red-300 text-sm hidden"></div>
        </div>

        <p id="songTitleOverlay" class="hidden">재생할 음악을 선택해주세요.</p>
        <div id="loadingIndicatorOverlay" class="hidden">
            <svg class="animate-spin h-5 w-5 mr-3 inline-block text-white" viewBox="0 0 24 24">
                <circle class="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" stroke-width="4"></circle>
                <path class="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path>
            </svg>
            음악 로딩 중...
        </div>
        <div id="errorMessageOverlay" class="text-red-300 hidden"></div>

        <button id="playButton">
            <img src="{play_button_image_url}" alt="Play Music">
        </button>

        <audio id="audioPlayer" class="hidden"></audio> <script>
            const playButton = document.getElementById('playButton');
            const audioPlayer = document.getElementById('audioPlayer');
            // 기존 #currentSong 대신 #songTitleOverlay 사용
            const songTitleOverlay = document.getElementById('songTitleOverlay');
            // 기존 로딩/에러 대신 오버레이 버전 사용
            const loadingIndicatorOverlay = document.getElementById('loadingIndicatorOverlay');
            const errorMessageOverlay = document.getElementById('errorMessageOverlay');


            playButton.addEventListener('click', async () => {{
                loadingIndicatorOverlay.classList.remove('hidden');
                errorMessageOverlay.classList.add('hidden');
                songTitleOverlay.textContent = '음악을 불러오는 중...';
                songTitleOverlay.classList.remove('hidden'); // 제목 오버레이 표시
                audioPlayer.pause();

                try {{
                    const response = await fetch('/random_music');
                    if (!response.ok) {{
                        throw new Error(`서버 오류: ${{response.status}} ${{response.statusText}}`);
                    }}
                    const data = await response.json();

                    if (data.error) {{
                        throw new Error(data.error);
                    }}

                    const musicUrl = data.music_url;
                    const musicTitle = data.music_title;

                    audioPlayer.src = musicUrl;
                    songTitleOverlay.textContent = `${{musicTitle}}`;

                    const playPromise = audioPlayer.play();
                    if (playPromise !== undefined) {{
                        playPromise.then(_ => {{
                            loadingIndicatorOverlay.classList.add('hidden');
                        }}).catch(error => {{
                            console.error('자동 재생 실패:', error);
                            loadingIndicatorOverlay.classList.add('hidden');
                            errorMessageOverlay.textContent = '음악 재생에 실패했습니다. (브라우저 정책) 다시 시도해주세요.';
                            errorMessageOverlay.classList.remove('hidden');
                        }});
                    }} else {{
                        loadingIndicatorOverlay.classList.add('hidden');
                    }}

                }} catch (error) {{
                    console.error('음악을 가져오거나 재생하는 중 오류 발생:', error);
                    loadingIndicatorOverlay.classList.add('hidden');
                    errorMessageOverlay.textContent = `오류: ${{error.message}}`;
                    errorMessageOverlay.classList.remove('hidden');
                }}
            }});

            audioPlayer.addEventListener('error', (e) => {{
                loadingIndicatorOverlay.classList.add('hidden');
                let errorMsg = '알 수 없는 오디오 재생 오류가 발생했습니다.';
                switch (e.target.error.code) {{
                    case e.target.error.MEDIA_ERR_ABORTED:
                        errorMsg = '음악 로딩이 중단되었습니다.';
                        break;
                    case e.target.error.MEDIA_ERR_NETWORK:
                        errorMsg = '네트워크 오류로 음악을 다운로드할 수 없습니다.';
                        break;
                    case e.target.error.MEDIA_ERR_DECODE:
                        errorMsg = '음악 파일을 디코딩할 수 없습니다 (손상되었거나 지원되지 않는 형식일 수 있습니다).';
                        break;
                    case e.target.error.MEDIA_ERR_SRC_NOT_SUPPORTED:
                        errorMsg = '음악 파일을 찾을 수 없거나 브라우저에서 지원하지 않는 형식입니다.';
                        break;
                    default:
                        errorMsg = `오디오 오류 코드: ${{e.target.error.code}}`;
                }}
                errorMessageOverlay.textContent = `오디오 오류: ${{errorMsg}}`;
                errorMessageOverlay.classList.remove('hidden');
                console.error('Audio playback error:', e.target.error);
            }});
        </script>
    </body>
    </html>
    '''.format(bg_image_url=bg_image_url, play_button_image_url=play_button_image_url)

@app.route('/random_music')
def get_random_music():
    print(f"'/random_music' 요청 수신. 현재 로드된 음악 파일: {music_files}") # 디버깅 추가
    if not music_files:
        print("오류: music_files 리스트가 비어 있습니다. 500 에러 반환.") # 디버깅 추가
        return jsonify({"error": "음악 파일이 없습니다. 폴더 경로와 파일 존재 여부를 확인해주세요."}), 500

    random_filename = random.choice(music_files)
    encoded_filename = urllib.parse.quote(random_filename)
    music_url = f"/music/{encoded_filename}"
    music_title = os.path.splitext(random_filename)[0]
    print(f"선택된 음악 파일: {random_filename}, URL: {music_url}") # 디버깅 추가

    return jsonify({"music_url": music_url, "music_title": music_title})

@app.route('/music/<path:filename>')
def download_file(filename):
    decoded_filename = urllib.parse.unquote(filename)
    full_path = os.path.join(folder_path, decoded_filename)
    print(f"음악 파일 요청: {decoded_filename}, 전체 경로: {full_path}") # 디버깅 추가
    if not os.path.exists(full_path):
        print(f"클라이언트 요청 파일 없음: {full_path}")
        return "File not found", 404
    return send_from_directory(folder_path, decoded_filename)

# Google Drive의 이미지를 제공하기 위한 새로운 엔드포인트
@app.route('/static_image/<path:filename>')
def serve_static_image(filename):
    # '새 폴더'가 MyDrive 바로 아래에 있다고 가정합니다.
    image_folder_path = '/content/drive/MyDrive/새 폴더/'
    image_full_path = os.path.join(image_folder_path, filename)
    print(f"정적 이미지 요청: {filename}, 전체 경로: {image_full_path}") # 디버깅 추가
    if not os.path.exists(image_full_path):
        print(f"요청된 이미지 파일 없음: {image_full_path}")
        return "Image not found", 404
    return send_from_directory(image_folder_path, filename)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Google Drive 마운트 중...
Mounted at /content/drive
Google Drive 마운트 완료.
ngrok 실행 파일이 이미 존재합니다.
ngrok 인증 토큰 설정 중...
ngrok 인증 토큰 설정 완료.
ngrok 터널 시작 중 (pyngrok 사용)...
ngrok 터널 URL: https://99b3-34-106-109-70.ngrok-free.app

--- '/content/drive/MyDrive/새 폴더' 폴더 내 모든 파일: ['01 Sonata “Apres une lecture de Dante”.mp3', '02 Mephisto Waltz No. 1.mp3', '06 Etudes de Execution Transcendante- Vision.mp3', '02 Années de Pelerinage Book 1- Switzerland- Au lac de Wallenstadt.mp3', '05 Piano Sonata No. 4- I.mp3', "05 Montecchi e Capuletti, From The Ballet 'Romeo and Juliet'.mp3", "01 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- I Adagio.mp3", "02 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- II Allegretto.mp3", "03 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- III Presto agitato.mp3", "04 Piano Sonata In F Minor, Op. 57 ‘'Appassionata'- I Allegro assai.mp3", "05 Piano Sonata In F Minor, Op. 57 ‘'Appassionata'- II Andante con moto.mp3", '02 Piano Sonata In B Minor, Op

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 23:49:12] "GET / HTTP/1.1" 200 -


정적 이미지 요청: Gemini_Generated_Image_3rnjgm3rnjgm3rnj.png, 전체 경로: /content/drive/MyDrive/새 폴더/Gemini_Generated_Image_3rnjgm3rnjgm3rnj.png


INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 23:49:13] "GET /static_image/Gemini_Generated_Image_3rnjgm3rnjgm3rnj.png HTTP/1.1" 200 -


음악 파일 요청: 160504-salvador-dali-01.webp, 전체 경로: /content/drive/MyDrive/새 폴더/160504-salvador-dali-01.webp


INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 23:49:13] "GET /music/160504-salvador-dali-01.webp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 23:49:14] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
# pyngrok이 설치되어 있지 않은 경우를 대비하여 설치 명령어를 추가합니다.
# 런타임 재시작 시 모듈이 사라질 수 있으므로, 매번 실행되도록 합니다.
!pip install pyngrok

import os
from flask import Flask, send_from_directory, jsonify
import urllib.parse
import random
import subprocess
import time
import json
from pyngrok import ngrok, conf

# --- Google Drive 마운트 ---
try:
    from google.colab import drive
    print("Google Drive 마운트 중...")
    drive.mount('/content/drive')
    print("Google Drive 마운트 완료.")
except Exception as e:
    print(f"Google Drive 마운트 중 오류 발생: {e}")
    print("Google Colab 환경이 아니거나, 이미 마운트되어 있을 수 있습니다.")
    print("만약 Drive가 필요한 환경이라면, 수동으로 'from google.colab import drive; drive.mount('/content/drive')'를 실행해주세요.")

app = Flask(__name__)

# --- ngrok 실행 파일 다운로드 및 설정 ---
ngrok_path = "/usr/local/bin/ngrok"
if not os.path.exists(ngrok_path):
    print("ngrok 실행 파일 다운로드 중...")
    try:
        subprocess.run(["wget", "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip"], check=True, capture_output=True)
        subprocess.run(["unzip", "-o", "ngrok-stable-linux-amd64.zip"], check=True, capture_output=True)
        subprocess.run(["mv", "ngrok", ngrok_path], check=True, capture_output=True)
        subprocess.run(["chmod", "+x", ngrok_path], check=True, capture_output=True)
        print("ngrok 다운로드 및 설정 완료.")
    except Exception as e:
        print(f"ngrok 설정 중 오류 발생: {e}. ngrok 수동 설치가 필요할 수 있습니다.")
        ngrok_path = None
else:
    print("ngrok 실행 파일이 이미 존재합니다.")

# --- ngrok 인증 토큰 설정 (필수) ---
NGROK_AUTH_TOKEN = "2zH1ogBnCwMM25wNykF3dGIHZbv_AN1TLC5aiBfduuyLLi6w"

if NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "YOUR_NGROK_AUTH_TOKEN_HERE":
    try:
        print("ngrok 인증 토큰 설정 중...")
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        print("ngrok 인증 토큰 설정 완료.")
    except Exception as e:
        print(f"ngrok 인증 토큰 설정 중 오류 발생: {e}. 토큰을 다시 확인해주세요.")
else:
    print("경고: ngrok 인증 토큰이 설정되지 않았습니다. ngrok 터널이 작동하지 않을 수 있습니다.")
    print("https://dashboard.ngrok.com/get-started/your-authtoken 에서 토큰을 얻어 'NGROK_AUTH_TOKEN' 변수에 입력해주세요.")

# --- ngrok 터널 시작 및 URL 획득 (pyngrok 사용) ---
public_url = None
if ngrok_path and NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "YOUR_NGROK_AUTH_TOKEN_HERE":
    try:
        print("ngrok 터널 시작 중 (pyngrok 사용)...")
        conf.get_default().ngrok_path = ngrok_path
        http_tunnel = ngrok.connect(5000, "http")
        public_url = http_tunnel.public_url
        print(f"ngrok 터널 URL: {public_url}")
    except Exception as e:
        print(f"ngrok 터널 시작 중 오류 발생: {e}. ngrok 서비스에 문제가 있을 수 있습니다.")
        public_url = None
else:
    print("ngrok 실행 파일 또는 인증 토큰 문제로 터널을 시작할 수 없습니다.")

# --- 사용자 설정: Google Drive에 마운트된 폴더 경로를 여기에 입력하세요 ---
folder_path = '/content/drive/MyDrive/새 폴더' # 음악 파일 경로
# -------------------------------------------------------------------------

# --- 사용자 설정: 배경 이미지 파일명 (Google Drive 폴더 내) ---
background_image_filename = '160504-salvador-dali-01.webp' # 배경 이미지 파일
# -----------------------------------------------------------------

music_files = []
all_files_in_folder = []

# Flask 앱 시작 전 파일 존재 여부 및 목록 로드
try:
    if not os.path.exists(folder_path):
        print(f"오류: 지정된 폴더 경로 '{folder_path}'가 존재하지 않습니다.")
        print("Google Drive 마운트 상태와 폴더 경로를 다시 확인해주세요.")
    else:
        all_files_in_folder = os.listdir(folder_path)
        print(f"\n--- '{folder_path}' 폴더 내 모든 파일: {all_files_in_folder} ---")

        for filename in all_files_in_folder:
            # MP3 파일만 추가하도록 확장자 확인
            if filename.lower().endswith(".mp3"): # 대소문자 구분 없이 .mp3 확인
                music_files.append(filename)
        print(f"--- {len(music_files)}개의 MP3 음악 파일이 로드되었습니다. 로드된 파일: {music_files} ---")

        if not music_files:
            print("경고: 지정된 폴더에 MP3 파일이 없습니다. 음악이 재생되지 않을 수 있습니다.")
            print("음악 파일이 실제로 이 경로에 있는지, 확장자가 .mp3인지 확인해주세요.")

        if background_image_filename not in all_files_in_folder:
            print(f"경고: 배경 이미지 파일 '{background_image_filename}'이(가) '{folder_path}'에 없습니다.")
            print("파일명을 정확히 확인하고 (대소문자 구분), 해당 폴더에 파일을 업로드해주세요.")
except Exception as e:
    print(f"음악 파일 로드 중 치명적인 오류 발생: {e}")
    music_files = [] # 오류 발생 시 목록 초기화

@app.route('/')
def home():
    """
    이 경로는 웹 애플리케이션의 메인 HTML 페이지를 제공합니다.
    사용자 인터페이스 (버튼, 음악 제목 표시 등) 및 JavaScript 로직이 포함됩니다.
    """
    # 배경 이미지 파일명을 URL 인코딩합니다.
    encoded_bg_filename = urllib.parse.quote(background_image_filename)
    # 배경 이미지의 URL 경로를 생성합니다.
    bg_image_url = f"/music/{encoded_bg_filename}"

    # Google Drive에 저장한 재생 버튼 이미지 경로를 '새 폴더' 내부로 지정합니다.
    # 이 경로가 정확한지 다시 한번 확인해주세요.
    play_button_image_path = '/content/drive/MyDrive/새 폴더/Gemini_Generated_Image_3rnjgm3rnjgm3rnj.png'

    # Flask에서 정적 파일로 제공하기 위한 URL 경로를 생성합니다.
    encoded_play_button_filename = urllib.parse.quote(os.path.basename(play_button_image_path))
    play_button_image_url = f"/static_image/{encoded_play_button_filename}"


    # HTML/CSS/JavaScript 내의 모든 리터럴 {}는 {{}}로 이스케이프합니다.
    return '''
    <!DOCTYPE html>
    <html lang="ko">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Beethoven 음악 재생기</title>
        <script src="https://cdn.tailwindcss.com"></script>
        <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap" rel="stylesheet">
        <style>
            body {{
                font-family: 'Inter', sans-serif;
                background-image: url('{bg_image_url}');
                background-size: cover;
                background-position: center;
                background-attachment: fixed;
                background-color: #1a202c;
                position: relative;
                overflow: hidden; /* 스크롤바 방지 */
            }}
            /* 메인 컨테이너 (이전 'Beethoven' 박스) - 완전히 숨김 */
            #app {{
                display: none; /* 이 요소를 완전히 숨깁니다 */
            }}

            /* 재생 버튼 스타일 (이미지 사용, 오른쪽 적당한 곳으로 조정) */
            #playButton {{
                position: absolute;
                bottom: 50px; /* 아래에서 50px 위로 */
                right: 50px; /* 오른쪽에서 50px 왼쪽으로 */
                width: 150px; /* 이미지 크기 조절 */
                height: 150px; /* 이미지 크기 조절 */
                background: none; /* 배경색 제거 */
                border: none;
                cursor: pointer;
                padding: 0;
                border-radius: 20%; /* 사각형 이미지일 경우 약간 둥글게 */
                overflow: hidden; /* 이미지가 버튼 영역을 넘치지 않도록 */
                box-shadow: 0 8px 25px rgba(0, 0, 0, 0.4); /* 그림자 강화 */
                transition: transform 0.2s ease-in-out, box-shadow 0.2s ease-in-out;
                z-index: 100; /* 다른 요소 위에 표시 */
                opacity: 0.7; /* 불투명도 70% */
            }}
            #playButton:hover {{
                transform: scale(1.08); /* 호버 시 약간 확대 */
                box-shadow: 0 12px 30px rgba(0, 0, 0, 0.5); /* 호버 시 그림자 더 강화 */
            }}
            #playButton:active {{
                transform: scale(0.95);
            }}
            #playButton img {{
                display: block;
                width: 100%;
                height: 100%;
                object-fit: cover; /* 이미지가 버튼 영역을 꽉 채우도록 */
                border-radius: inherit; /* 부모의 border-radius 상속 */
            }}
             /* 음악 제목 표시를 위한 새로운 요소 (버튼 위에 띄울 수 있음) */
            #songTitleOverlay {{
                position: absolute;
                top: 20px; /* 상단에서 20px 아래로 */
                left: 50%;
                transform: translateX(-50%);
                color: white;
                font-size: 1.5rem;
                font-weight: bold;
                text-shadow: 2px 2px 5px rgba(0,0,0,0.7);
                z-index: 10;
                opacity: 0.7; /* 불투명도 70% */
            }}
            #loadingIndicatorOverlay, #errorMessageOverlay {{
                position: absolute;
                top: 80px; /* 제목 아래에 표시 */
                left: 50%;
                transform: translateX(-50%);
                z-index: 10;
                color: white; /* 로딩/에러 메시지 색상 */
                background-color: rgba(0,0,0,0.5);
                padding: 5px 10px;
                border-radius: 5px;
            }}
        </style>
    </head>
    <body class="min-h-screen flex items-center justify-center p-4">
        <div id="app">
            <h1 class="text-4xl md:text-5xl font-extrabold mb-6 drop-shadow-lg">Beethoven</h1>
            <p id="currentSong" class="text-lg md:text-xl font-medium mb-8 opacity-90 min-h-[2.5rem] flex items-center justify-center">
                재생할 음악을 선택해주세요.
            </p>
            <audio id="audioPlayer" class="hidden"></audio>
            <div id="loadingIndicator" class="mt-4 text-sm opacity-70 hidden">
                <svg class="animate-spin h-5 w-5 mr-3 inline-block text-white" viewBox="0 0 24 24">
                    <circle class="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" stroke-width="4"></circle>
                    <path class="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path>
                </svg>
                음악 로딩 중...
            </div>
            <div id="errorMessage" class="mt-4 text-red-300 text-sm hidden"></div>
        </div>

        <p id="songTitleOverlay" class="hidden">재생할 음악을 선택해주세요.</p>
        <div id="loadingIndicatorOverlay" class="hidden">
            <svg class="animate-spin h-5 w-5 mr-3 inline-block text-white" viewBox="0 0 24 24">
                <circle class="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" stroke-width="4"></circle>
                <path class="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path>
            </svg>
            음악 로딩 중...
        </div>
        <div id="errorMessageOverlay" class="text-red-300 hidden"></div>

        <button id="playButton">
            <img src="{play_button_image_url}" alt="Play Music">
        </button>

        <audio id="audioPlayer" class="hidden"></audio> <script>
            const playButton = document.getElementById('playButton');
            const audioPlayer = document.getElementById('audioPlayer');
            // 기존 #currentSong 대신 #songTitleOverlay 사용
            const songTitleOverlay = document.getElementById('songTitleOverlay');
            // 기존 로딩/에러 대신 오버레이 버전 사용
            const loadingIndicatorOverlay = document.getElementById('loadingIndicatorOverlay');
            const errorMessageOverlay = document.getElementById('errorMessageOverlay');


            playButton.addEventListener('click', async () => {{
                loadingIndicatorOverlay.classList.remove('hidden');
                errorMessageOverlay.classList.add('hidden');
                songTitleOverlay.textContent = '음악을 불러오는 중...';
                songTitleOverlay.classList.remove('hidden'); // 제목 오버레이 표시
                audioPlayer.pause();

                try {{
                    const response = await fetch('/random_music');
                    if (!response.ok) {{
                        throw new Error(`서버 오류: ${{response.status}} ${{response.statusText}}`);
                    }}
                    const data = await response.json();

                    if (data.error) {{
                        throw new Error(data.error);
                    }}

                    const musicUrl = data.music_url;
                    const musicTitle = data.music_title;

                    audioPlayer.src = musicUrl;
                    songTitleOverlay.textContent = `${{musicTitle}}`;

                    const playPromise = audioPlayer.play();
                    if (playPromise !== undefined) {{
                        playPromise.then(_ => {{
                            loadingIndicatorOverlay.classList.add('hidden');
                        }}).catch(error => {{
                            console.error('자동 재생 실패:', error);
                            loadingIndicatorOverlay.classList.add('hidden');
                            errorMessageOverlay.textContent = '음악 재생에 실패했습니다. (브라우저 정책) 다시 시도해주세요.';
                            errorMessageOverlay.classList.remove('hidden');
                        }});
                    }} else {{
                        loadingIndicatorOverlay.classList.add('hidden');
                    }}

                }} catch (error) {{
                    console.error('음악을 가져오거나 재생하는 중 오류 발생:', error);
                    loadingIndicatorOverlay.classList.add('hidden');
                    errorMessageOverlay.textContent = `오류: ${{error.message}}`;
                    errorMessageOverlay.classList.remove('hidden');
                }}
            }});

            audioPlayer.addEventListener('error', (e) => {{
                loadingIndicatorOverlay.classList.add('hidden');
                let errorMsg = '알 수 없는 오디오 재생 오류가 발생했습니다.';
                switch (e.target.error.code) {{
                    case e.target.error.MEDIA_ERR_ABORTED:
                        errorMsg = '음악 로딩이 중단되었습니다.';
                        break;
                    case e.target.error.MEDIA_ERR_NETWORK:
                        errorMsg = '네트워크 오류로 음악을 다운로드할 수 없습니다.';
                        break;
                    case e.target.error.MEDIA_ERR_DECODE:
                        errorMsg = '음악 파일을 디코딩할 수 없습니다 (손상되었거나 지원되지 않는 형식일 수 있습니다).';
                        break;
                    case e.target.error.MEDIA_ERR_SRC_NOT_SUPPORTED:
                        errorMsg = '음악 파일을 찾을 수 없거나 브라우저에서 지원하지 않는 형식입니다.';
                        break;
                    default:
                        errorMsg = `오디오 오류 코드: ${{e.target.error.code}}`;
                }}
                errorMessageOverlay.textContent = `오디오 오류: ${{errorMsg}}`;
                errorMessageOverlay.classList.remove('hidden');
                console.error('Audio playback error:', e.target.error);
            }});
        </script>
    </body>
    </html>
    '''.format(bg_image_url=bg_image_url, play_button_image_url=play_button_image_url)

@app.route('/random_music')
def get_random_music():
    print(f"'/random_music' 요청 수신. 현재 로드된 음악 파일: {music_files}") # 디버깅 추가
    if not music_files:
        print("오류: music_files 리스트가 비어 있습니다. 500 에러 반환.") # 디버깅 추가
        return jsonify({"error": "음악 파일이 없습니다. 폴더 경로와 파일 존재 여부를 확인해주세요."}), 500

    random_filename = random.choice(music_files)
    encoded_filename = urllib.parse.quote(random_filename)
    music_url = f"/music/{encoded_filename}"
    music_title = os.path.splitext(random_filename)[0]
    print(f"선택된 음악 파일: {random_filename}, URL: {music_url}") # 디버깅 추가

    return jsonify({"music_url": music_url, "music_title": music_title})

@app.route('/music/<path:filename>')
def download_file(filename):
    decoded_filename = urllib.parse.unquote(filename)
    full_path = os.path.join(folder_path, decoded_filename)
    print(f"음악 파일 요청: {decoded_filename}, 전체 경로: {full_path}") # 디버깅 추가
    if not os.path.exists(full_path):
        print(f"클라이언트 요청 파일 없음: {full_path}")
        return "File not found", 404
    return send_from_directory(folder_path, decoded_filename)

# Google Drive의 이미지를 제공하기 위한 새로운 엔드포인트
@app.route('/static_image/<path:filename>')
def serve_static_image(filename):
    # '새 폴더'가 MyDrive 바로 아래에 있다고 가정합니다.
    image_folder_path = '/content/drive/MyDrive/새 폴더/'
    image_full_path = os.path.join(image_folder_path, filename)
    print(f"정적 이미지 요청: {filename}, 전체 경로: {image_full_path}") # 디버깅 추가
    if not os.path.exists(image_full_path):
        print(f"요청된 이미지 파일 없음: {image_full_path}")
        return "Image not found", 404
    return send_from_directory(image_folder_path, filename)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Google Drive 마운트 중...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive 마운트 완료.
ngrok 실행 파일이 이미 존재합니다.
ngrok 인증 토큰 설정 중...
ngrok 인증 토큰 설정 완료.
ngrok 터널 시작 중 (pyngrok 사용)...
ngrok 터널 URL: https://89a1-34-106-135-216.ngrok-free.app

--- '/content/drive/MyDrive/새 폴더' 폴더 내 모든 파일: ['01 Sonata “Apres une lecture de Dante”.mp3', '02 Mephisto Waltz No. 1.mp3', '06 Etudes de Execution Transcendante- Vision.mp3', '02 Années de Pelerinage Book 1- Switzerland- Au lac de Wallenstadt.mp3', '05 Piano Sonata No. 4- I.mp3', "05 Montecchi e Capuletti, From The Ballet 'Romeo and Juliet'.mp3", "01 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- I Adagio.mp3", "02 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- II Allegretto.mp3", "03 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- III Presto agitato.mp3", "04 Piano Sonata In F Minor, Op. 57 ‘'Appassionata'- I Allegro assai.mp3", "05 Pi

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 09:07:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 09:07:01] "GET /static_image/Gemini_Generated_Image_3rnjgm3rnjgm3rnj.png HTTP/1.1" 200 -


정적 이미지 요청: Gemini_Generated_Image_3rnjgm3rnjgm3rnj.png, 전체 경로: /content/drive/MyDrive/새 폴더/Gemini_Generated_Image_3rnjgm3rnjgm3rnj.png


INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 09:07:02] "GET /music/160504-salvador-dali-01.webp HTTP/1.1" 200 -


음악 파일 요청: 160504-salvador-dali-01.webp, 전체 경로: /content/drive/MyDrive/새 폴더/160504-salvador-dali-01.webp


...

INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 10:22:51] "GET /music/01%20Piano%20Sonata%20in%20B%20Minor,%20Op.%2040,%20No.%202.mp3 HTTP/1.1" 206 -


음악 파일 요청: 01 Piano Sonata in B Minor, Op. 40, No. 2.mp3, 전체 경로: /content/drive/MyDrive/새 폴더/01 Piano Sonata in B Minor, Op. 40, No. 2.mp3


INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 10:22:56] "GET /random_music HTTP/1.1" 200 -


'/random_music' 요청 수신. 현재 로드된 음악 파일: ['01 Sonata “Apres une lecture de Dante”.mp3', '02 Mephisto Waltz No. 1.mp3', '06 Etudes de Execution Transcendante- Vision.mp3', '02 Années de Pelerinage Book 1- Switzerland- Au lac de Wallenstadt.mp3', '05 Piano Sonata No. 4- I.mp3', "05 Montecchi e Capuletti, From The Ballet 'Romeo and Juliet'.mp3", "01 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- I Adagio.mp3", "02 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- II Allegretto.mp3", "03 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- III Presto agitato.mp3", "04 Piano Sonata In F Minor, Op. 57 ‘'Appassionata'- I Allegro assai.mp3", "05 Piano Sonata In F Minor, Op. 57 ‘'Appassionata'- II Andante con moto.mp3", '02 Piano Sonata In B Minor, Op. 40, No. 2- I Molto adagio e sostenuto-allegro con fuoco.mp3', '03 Piano Sonata In B Minor, Op. 40, No. 2- II Largo, mesto e patetico-allegro.mp3', '04 Piano Sonata In G Major, Hob. XVI, No. 27- I Allegro con brio.mp3', '01 Pi

INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 10:22:57] "GET /music/06%20Etudes%20de%20Execution%20Transcendante-%20Vision.mp3 HTTP/1.1" 206 -


In [ ]:
# pyngrok이 설치되어 있지 않은 경우를 대비하여 설치 명령어를 추가합니다.
# 런타임 재시작 시 모듈이 사라질 수 있으므로, 매번 실행되도록 합니다.
!pip install pyngrok

import os
from flask import Flask, send_from_directory, jsonify
import urllib.parse
import random
import subprocess
import time
import json
from pyngrok import ngrok, conf

# --- Google Drive 마운트 ---
try:
    from google.colab import drive
    print("Google Drive 마운트 중...")
    drive.mount('/content/drive')
    print("Google Drive 마운트 완료.")
except Exception as e:
    print(f"Google Drive 마운트 중 오류 발생: {e}")
    print("Google Colab 환경이 아니거나, 이미 마운트되어 있을 수 있습니다.")
    print("만약 Drive가 필요한 환경이라면, 수동으로 'from google.colab import drive; drive.mount('/content/drive')'를 실행해주세요.")

app = Flask(__name__)

# --- ngrok 실행 파일 다운로드 및 설정 ---
ngrok_path = "/usr/local/bin/ngrok"
if not os.path.exists(ngrok_path):
    print("ngrok 실행 파일 다운로드 중...")
    try:
        subprocess.run(["wget", "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip"], check=True, capture_output=True)
        subprocess.run(["unzip", "-o", "ngrok-stable-linux-amd64.zip"], check=True, capture_output=True)
        subprocess.run(["mv", "ngrok", ngrok_path], check=True, capture_output=True)
        subprocess.run(["chmod", "+x", ngrok_path], check=True, capture_output=True)
        print("ngrok 다운로드 및 설정 완료.")
    except Exception as e:
        print(f"ngrok 설정 중 오류 발생: {e}. ngrok 수동 설치가 필요할 수 있습니다.")
        ngrok_path = None
else:
    print("ngrok 실행 파일이 이미 존재합니다.")

# --- ngrok 인증 토큰 설정 (필수) ---
NGROK_AUTH_TOKEN = "2zH1ogBnCwMM25wNykF3dGIHZbv_AN1TLC5aiBfduuyLLi6w"

if NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "YOUR_NGROK_AUTH_TOKEN_HERE":
    try:
        print("ngrok 인증 토큰 설정 중...")
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        print("ngrok 인증 토큰 설정 완료.")
    except Exception as e:
        print(f"ngrok 인증 토큰 설정 중 오류 발생: {e}. 토큰을 다시 확인해주세요.")
else:
    print("경고: ngrok 인증 토큰이 설정되지 않았습니다. ngrok 터널이 작동하지 않을 수 있습니다.")
    print("https://dashboard.ngrok.com/get-started/your-authtoken 에서 토큰을 얻어 'NGROK_AUTH_TOKEN' 변수에 입력해주세요.")

# --- ngrok 터널 시작 및 URL 획득 (pyngrok 사용) ---
public_url = None
if ngrok_path and NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "YOUR_NGROK_AUTH_TOKEN_HERE":
    try:
        print("ngrok 터널 시작 중 (pyngrok 사용)...")
        conf.get_default().ngrok_path = ngrok_path
        http_tunnel = ngrok.connect(5000, "http")
        public_url = http_tunnel.public_url
        print(f"ngrok 터널 URL: {public_url}")
    except Exception as e:
        print(f"ngrok 터널 시작 중 오류 발생: {e}. ngrok 서비스에 문제가 있을 수 있습니다.")
        public_url = None
else:
    print("ngrok 실행 파일 또는 인증 토큰 문제로 터널을 시작할 수 없습니다.")

# --- 사용자 설정: Google Drive에 마운트된 폴더 경로를 여기에 입력하세요 ---
folder_path = '/content/drive/MyDrive/새 폴더' # 음악 파일 경로
# -------------------------------------------------------------------------

# --- 사용자 설정: 배경 이미지 파일명 (Google Drive 폴더 내) ---
background_image_filename = '160504-salvador-dali-01.webp' # 배경 이미지 파일
# -----------------------------------------------------------------

music_files = []
all_files_in_folder = []

# Flask 앱 시작 전 파일 존재 여부 및 목록 로드
try:
    if not os.path.exists(folder_path):
        print(f"오류: 지정된 폴더 경로 '{folder_path}'가 존재하지 않습니다.")
        print("Google Drive 마운트 상태와 폴더 경로를 다시 확인해주세요.")
    else:
        all_files_in_folder = os.listdir(folder_path)
        print(f"\n--- '{folder_path}' 폴더 내 모든 파일: {all_files_in_folder} ---")

        for filename in all_files_in_folder:
            # MP3 파일만 추가하도록 확장자 확인
            if filename.lower().endswith(".mp3"): # 대소문자 구분 없이 .mp3 확인
                music_files.append(filename)
        print(f"--- {len(music_files)}개의 MP3 음악 파일이 로드되었습니다. 로드된 파일: {music_files} ---")

        if not music_files:
            print("경고: 지정된 폴더에 MP3 파일이 없습니다. 음악이 재생되지 않을 수 있습니다.")
            print("음악 파일이 실제로 이 경로에 있는지, 확장자가 .mp3인지 확인해주세요.")

        if background_image_filename not in all_files_in_folder:
            print(f"경고: 배경 이미지 파일 '{background_image_filename}'이(가) '{folder_path}'에 없습니다.")
            print("파일명을 정확히 확인하고 (대소문자 구분), 해당 폴더에 파일을 업로드해주세요.")
except Exception as e:
    print(f"음악 파일 로드 중 치명적인 오류 발생: {e}")
    music_files = [] # 오류 발생 시 목록 초기화

@app.route('/')
def home():
    """
    이 경로는 웹 애플리케이션의 메인 HTML 페이지를 제공합니다.
    사용자 인터페이스 (버튼, 음악 제목 표시 등) 및 JavaScript 로직이 포함됩니다.
    """
    # 배경 이미지 파일명을 URL 인코딩합니다.
    encoded_bg_filename = urllib.parse.quote(background_image_filename)
    # 배경 이미지의 URL 경로를 생성합니다.
    bg_image_url = f"/music/{encoded_bg_filename}"

    # Google Drive에 저장한 재생 버튼 이미지 경로를 '새 폴더' 내부로 지정합니다.
    # 이 경로가 정확한지 다시 한번 확인해주세요.
    play_button_image_path = '/content/drive/MyDrive/새 폴더/Gemini_Generated_Image_3rnjgm3rnjgm3rnj.png'

    # Flask에서 정적 파일로 제공하기 위한 URL 경로를 생성합니다.
    encoded_play_button_filename = urllib.parse.quote(os.path.basename(play_button_image_path))
    play_button_image_url = f"/static_image/{encoded_play_button_filename}"


    # HTML/CSS/JavaScript 내의 모든 리터럴 {}는 {{}}로 이스케이프합니다.
    return '''
    <!DOCTYPE html>
    <html lang="ko">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Beethoven 음악 재생기</title>
        <script src="https://cdn.tailwindcss.com"></script>
        <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap" rel="stylesheet">
        <style>
            body {{
                font-family: 'Inter', sans-serif;
                background-image: url('{bg_image_url}');
                background-size: cover;
                background-position: center;
                background-attachment: fixed;
                background-color: #1a202c;
                position: relative;
                overflow: hidden; /* 스크롤바 방지 */
            }}
            /* 메인 컨테이너 (이전 'Beethoven' 박스) - 완전히 숨김 */
            #app {{
                display: none; /* 이 요소를 완전히 숨깁니다 */
            }}

            /* 재생 버튼 스타일 (이미지 사용, 오른쪽 적당한 곳으로 조정) */
            #playButton {{
                position: absolute;
                bottom: 50px; /* 아래에서 50px 위로 */
                right: 50px; /* 오른쪽에서 50px 왼쪽으로 */
                width: 150px; /* 이미지 크기 조절 */
                height: 150px; /* 이미지 크기 조절 */
                background: none; /* 배경색 제거 */
                border: none;
                cursor: pointer;
                padding: 0;
                border-radius: 20%; /* 사각형 이미지일 경우 약간 둥글게 */
                overflow: hidden; /* 이미지가 버튼 영역을 넘치지 않도록 */
                box-shadow: 0 8px 25px rgba(0, 0, 0, 0.4); /* 그림자 강화 */
                transition: transform 0.2s ease-in-out, box-shadow 0.2s ease-in-out;
                z-index: 100; /* 다른 요소 위에 표시 */
                opacity: 0.7; /* 불투명도 70% */
            }}
            #playButton:hover {{
                transform: scale(1.08); /* 호버 시 약간 확대 */
                box-shadow: 0 12px 30px rgba(0, 0, 0, 0.5); /* 호버 시 그림자 더 강화 */
            }}
            #playButton:active {{
                transform: scale(0.95);
            }}
            #playButton img {{
                display: block;
                width: 100%;
                height: 100%;
                object-fit: cover; /* 이미지가 버튼 영역을 꽉 채우도록 */
                border-radius: inherit; /* 부모의 border-radius 상속 */
            }}
             /* 음악 제목 표시를 위한 새로운 요소 (왼쪽 상단으로 이동 및 너비 조절) */
            #songTitleOverlay {{
                position: absolute;
                top: 50px; /* 상단에서 50px 아래로 */
                left: 50px; /* 왼쪽에서 50px 오른쪽으로 */
                color: white;
                font-size: 1.5rem;
                font-weight: bold;
                text-shadow: 2px 2px 5px rgba(0,0,0,0.7);
                z-index: 10;
                opacity: 0.7; /* 불투명도 70% */
                max-width: 400px; /* 최대 너비 설정 (줄바꿈 유도) */
                text-align: left; /* 텍스트 왼쪽 정렬 */
                background-color: rgba(0,0,0,0.3); /* 배경색 추가 (약간 투명한 검은색) */
                padding: 10px 15px; /* 패딩 추가 */
                border-radius: 8px; /* 모서리 둥글게 */
                box-sizing: border-box; /* 패딩이 너비에 포함되도록 */
            }}
            #loadingIndicatorOverlay, #errorMessageOverlay {{
                position: absolute;
                top: 150px; /* 제목 아래에 표시 (제목 위치 변경에 맞춰 조정) */
                left: 50px; /* 노래 제목과 같은 왼쪽 정렬 */
                z-index: 10;
                color: white; /* 로딩/에러 메시지 색상 */
                background-color: rgba(0,0,0,0.5);
                padding: 5px 10px;
                border-radius: 5px;
            }}
        </style>
    </head>
    <body class="min-h-screen flex items-center justify-center p-4">
        <div id="app">
            <h1 class="text-4xl md:text-5xl font-extrabold mb-6 drop-shadow-lg">Beethoven</h1>
            <p id="currentSong" class="text-lg md:text-xl font-medium mb-8 opacity-90 min-h-[2.5rem] flex items-center justify-center">
                재생할 음악을 선택해주세요.
            </p>
            <audio id="audioPlayer" class="hidden"></audio>
            <div id="loadingIndicator" class="mt-4 text-sm opacity-70 hidden">
                <svg class="animate-spin h-5 w-5 mr-3 inline-block text-white" viewBox="0 0 24 24">
                    <circle class="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" stroke-width="4"></circle>
                    <path class="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path>
                </svg>
                음악 로딩 중...
            </div>
            <div id="errorMessage" class="mt-4 text-red-300 text-sm hidden"></div>
        </div>

        <p id="songTitleOverlay" class="hidden">재생할 음악을 선택해주세요.</p>
        <div id="loadingIndicatorOverlay" class="hidden">
            <svg class="animate-spin h-5 w-5 mr-3 inline-block text-white" viewBox="0 0 24 24">
                <circle class="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" stroke-width="4"></circle>
                <path class="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path>
            </svg>
            음악 로딩 중...
        </div>
        <div id="errorMessageOverlay" class="text-red-300 hidden"></div>

        <button id="playButton">
            <img src="{play_button_image_url}" alt="Play Music">
        </button>

        <audio id="audioPlayer" class="hidden"></audio>
        <script>
            const playButton = document.getElementById('playButton');
            const audioPlayer = document.getElementById('audioPlayer');
            // 기존 #currentSong 대신 #songTitleOverlay 사용
            const songTitleOverlay = document.getElementById('songTitleOverlay');
            // 기존 로딩/에러 대신 오버레이 버전 사용
            const loadingIndicatorOverlay = document.getElementById('loadingIndicatorOverlay');
            const errorMessageOverlay = document.getElementById('errorMessageOverlay');


            playButton.addEventListener('click', async () => {{
                loadingIndicatorOverlay.classList.remove('hidden');
                errorMessageOverlay.classList.add('hidden');
                songTitleOverlay.textContent = '음악을 불러오는 중...';
                songTitleOverlay.classList.remove('hidden'); // 제목 오버레이 표시
                audioPlayer.pause();

                try {{
                    const response = await fetch('/random_music');
                    if (!response.ok) {{
                        throw new Error(`서버 오류: ${{response.status}} ${{response.statusText}}`);
                    }}
                    const data = await response.json();

                    if (data.error) {{
                        throw new Error(data.error);
                    }}

                    const musicUrl = data.music_url;
                    const musicTitle = data.music_title;

                    audioPlayer.src = musicUrl;
                    songTitleOverlay.textContent = `${{musicTitle}}`;

                    const playPromise = audioPlayer.play();
                    if (playPromise !== undefined) {{
                        playPromise.then(_ => {{
                            loadingIndicatorOverlay.classList.add('hidden');
                        }}).catch(error => {{
                            console.error('자동 재생 실패:', error);
                            loadingIndicatorOverlay.classList.add('hidden');
                            errorMessageOverlay.textContent = '음악 재생에 실패했습니다. (브라우저 정책) 다시 시도해주세요.';
                            errorMessageOverlay.classList.remove('hidden');
                        }});
                    }} else {{
                        loadingIndicatorOverlay.classList.add('hidden');
                    }}

                }} catch (error) {{
                    console.error('음악을 가져오거나 재생하는 중 오류 발생:', error);
                    loadingIndicatorOverlay.classList.add('hidden');
                    errorMessageOverlay.textContent = `오류: ${{error.message}}`;
                    errorMessageOverlay.classList.remove('hidden');
                }}
            }});

            audioPlayer.addEventListener('error', (e) => {{
                loadingIndicatorOverlay.classList.add('hidden');
                let errorMsg = '알 수 없는 오디오 재생 오류가 발생했습니다.';
                switch (e.target.error.code) {{
                    case e.target.error.MEDIA_ERR_ABORTED:
                        errorMsg = '음악 로딩이 중단되었습니다.';
                        break;
                    case e.target.error.MEDIA_ERR_NETWORK:
                        errorMsg = '네트워크 오류로 음악을 다운로드할 수 없습니다.';
                        break;
                    case e.target.error.MEDIA_ERR_DECODE:
                        errorMsg = '음악 파일을 디코딩할 수 없습니다 (손상되었거나 지원되지 않는 형식일 수 있습니다).';
                        break;
                    case e.target.error.MEDIA_ERR_SRC_NOT_SUPPORTED:
                        errorMsg = '음악 파일을 찾을 수 없거나 브라우저에서 지원하지 않는 형식입니다.';
                        break;
                    default:
                        errorMsg = `오디오 오류 코드: ${{e.target.error.code}}`;
                }}
                errorMessageOverlay.textContent = `오디오 오류: ${{errorMsg}}`;
                errorMessageOverlay.classList.remove('hidden');
                console.error('Audio playback error:', e.target.error);
            }});
        </script>
    </body>
    </html>
    '''.format(bg_image_url=bg_image_url, play_button_image_url=play_button_image_url)

@app.route('/random_music')
def get_random_music():
    print(f"'/random_music' 요청 수신. 현재 로드된 음악 파일: {music_files}") # 디버깅 추가
    if not music_files:
        print("오류: music_files 리스트가 비어 있습니다. 500 에러 반환.") # 디버깅 추가
        return jsonify({"error": "음악 파일이 없습니다. 폴더 경로와 파일 존재 여부를 확인해주세요."}), 500

    random_filename = random.choice(music_files)
    encoded_filename = urllib.parse.quote(random_filename)
    music_url = f"/music/{encoded_filename}"
    music_title = os.path.splitext(random_filename)[0]
    print(f"선택된 음악 파일: {random_filename}, URL: {music_url}") # 디버깅 추가

    return jsonify({"music_url": music_url, "music_title": music_title})

@app.route('/music/<path:filename>')
def download_file(filename):
    decoded_filename = urllib.parse.unquote(filename)
    full_path = os.path.join(folder_path, decoded_filename)
    print(f"음악 파일 요청: {decoded_filename}, 전체 경로: {full_path}") # 디버깅 추가
    if not os.path.exists(full_path):
        print(f"클라이언트 요청 파일 없음: {full_path}")
        return "File not found", 404
    return send_from_directory(folder_path, decoded_filename)

# Google Drive의 이미지를 제공하기 위한 새로운 엔드포인트
@app.route('/static_image/<path:filename>')
def serve_static_image(filename):
    # '새 폴더'가 MyDrive 바로 아래에 있다고 가정합니다.
    image_folder_path = '/content/drive/MyDrive/새 폴더/'
    image_full_path = os.path.join(image_folder_path, filename)
    print(f"정적 이미지 요청: {filename}, 전체 경로: {image_full_path}") # 디버깅 추가
    if not os.path.exists(image_full_path):
        print(f"요청된 이미지 파일 없음: {image_full_path}")
        return "Image not found", 404
    return send_from_directory(image_folder_path, filename)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Google Drive 마운트 중...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive 마운트 완료.
ngrok 실행 파일이 이미 존재합니다.
ngrok 인증 토큰 설정 중...
ngrok 인증 토큰 설정 완료.
ngrok 터널 시작 중 (pyngrok 사용)...
ngrok 터널 URL: https://562a-34-106-135-216.ngrok-free.app

--- '/content/drive/MyDrive/새 폴더' 폴더 내 모든 파일: ['01 Sonata “Apres une lecture de Dante”.mp3', '02 Mephisto Waltz No. 1.mp3', '06 Etudes de Execution Transcendante- Vision.mp3', '02 Années de Pelerinage Book 1- Switzerland- Au lac de Wallenstadt.mp3', '05 Piano Sonata No. 4- I.mp3', "05 Montecchi e Capuletti, From The Ballet 'Romeo and Juliet'.mp3", "01 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- I Adagio.mp3", "02 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- II Allegretto.mp3", "03 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- III Presto agitato.mp3", "04 Piano Sonata In F Minor, Op. 57 ‘'Appassionata'- I Allegro assai.mp3", "05 Pi

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 09:06:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 09:06:27] "GET /static_image/Gemini_Generated_Image_3rnjgm3rnjgm3rnj.png HTTP/1.1" 200 -


정적 이미지 요청: Gemini_Generated_Image_3rnjgm3rnjgm3rnj.png, 전체 경로: /content/drive/MyDrive/새 폴더/Gemini_Generated_Image_3rnjgm3rnjgm3rnj.png


INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 09:06:27] "GET /music/160504-salvador-dali-01.webp HTTP/1.1" 200 -


음악 파일 요청: 160504-salvador-dali-01.webp, 전체 경로: /content/drive/MyDrive/새 폴더/160504-salvador-dali-01.webp


INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 09:06:28] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 09:06:29] "GET /random_music HTTP/1.1" 200 -


'/random_music' 요청 수신. 현재 로드된 음악 파일: ['01 Sonata “Apres une lecture de Dante”.mp3', '02 Mephisto Waltz No. 1.mp3', '06 Etudes de Execution Transcendante- Vision.mp3', '02 Années de Pelerinage Book 1- Switzerland- Au lac de Wallenstadt.mp3', '05 Piano Sonata No. 4- I.mp3', "05 Montecchi e Capuletti, From The Ballet 'Romeo and Juliet'.mp3", "01 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- I Adagio.mp3", "02 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- II Allegretto.mp3", "03 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- III Presto agitato.mp3", "04 Piano Sonata In F Minor, Op. 57 ‘'Appassionata'- I Allegro assai.mp3", "05 Piano Sonata In F Minor, Op. 57 ‘'Appassionata'- II Andante con moto.mp3", '02 Piano Sonata In B Minor, Op. 40, No. 2- I Molto adagio e sostenuto-allegro con fuoco.mp3', '03 Piano Sonata In B Minor, Op. 40, No. 2- II Largo, mesto e patetico-allegro.mp3', '04 Piano Sonata In G Major, Hob. XVI, No. 27- I Allegro con brio.mp3', '01 Pi

INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 09:06:30] "GET /music/02%20Sonata%20No.%202%20in%20B-Flat%20Minor,%20Op.%2035%20'Funeral%20March'-%20II.%20Scherzo.mp3 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 09:06:30] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 09:06:36] "GET /random_music HTTP/1.1" 200 -


'/random_music' 요청 수신. 현재 로드된 음악 파일: ['01 Sonata “Apres une lecture de Dante”.mp3', '02 Mephisto Waltz No. 1.mp3', '06 Etudes de Execution Transcendante- Vision.mp3', '02 Années de Pelerinage Book 1- Switzerland- Au lac de Wallenstadt.mp3', '05 Piano Sonata No. 4- I.mp3', "05 Montecchi e Capuletti, From The Ballet 'Romeo and Juliet'.mp3", "01 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- I Adagio.mp3", "02 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- II Allegretto.mp3", "03 Piano Sonata In C-Sharp Minor, Op. 27, No. 2 'Moonlight'- III Presto agitato.mp3", "04 Piano Sonata In F Minor, Op. 57 ‘'Appassionata'- I Allegro assai.mp3", "05 Piano Sonata In F Minor, Op. 57 ‘'Appassionata'- II Andante con moto.mp3", '02 Piano Sonata In B Minor, Op. 40, No. 2- I Molto adagio e sostenuto-allegro con fuoco.mp3', '03 Piano Sonata In B Minor, Op. 40, No. 2- II Largo, mesto e patetico-allegro.mp3', '04 Piano Sonata In G Major, Hob. XVI, No. 27- I Allegro con brio.mp3', '01 Pi

INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 09:06:37] "GET /music/03%20Piano%20Sonata%20In%20C-Sharp%20Minor,%20Op.%2027,%20No.%202%20'Moonlight'-%20III%20Presto%20agitato.mp3 HTTP/1.1" 206 -
